In [1]:
import mne
from mne.decoding import CSP
import pandas
import numpy as np
#Using locs to read montage
montage_NE = mne.channels.read_montage(kind='Enobio32',path=r'F:\software\1NE_NIC2\1channelL')
epochs_zzl2Session1 = mne.io.read_epochs_eeglab(input_fname=r'F:\software\1NE_NIC2\raw\MI10Oct_and8_oneCmc\zzl2Session1.set',
                             montage=montage_NE)
# %matplotlib qt
# epochs_zzl2Session1.plot()
MI_epochs = epochs_zzl2Session1.copy()
MI_epochs.crop(tmin = 3, tmax = 7)
# MI_epochs.plot()

y = pandas.read_csv('F:/software/1NE_NIC2/raw/MI10Oct_and8_oneCmc/100times_test_section1.csv')
labels=y.iloc[1:101, 2]
y=np.array([])
for label in labels:
    if label == 'left':
        y=np.append(y,1)
    else:
        y=np.append(y,0)

X=MI_epochs.get_data()
csp = CSP(n_components=4,reg='shrunk',cov_est='concat',rank ='full')

csp.fit(X,y)

Extracting parameters from F:\software\1NE_NIC2\raw\MI10Oct_and8_oneCmc\zzl2Session1.set...
100 matching events found
No baseline correction applied
Not setting metadata


<ipython-input-1-984c54476614>:8: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  montage=montage_NE)


0 projection items activated
Ready.
Computing data rank from raw with rank='full'
    MAG: rank 32 from info
Reducing data rank from 32 -> 32
Estimating covariance using SHRUNK


F:\software\2PYTHON\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Done.
Computing data rank from raw with rank='full'
    MAG: rank 32 from info
Reducing data rank from 32 -> 32
Estimating covariance using SHRUNK


F:\software\2PYTHON\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Done.


CSP(cov_est='concat', cov_method_params=None, log=None, n_components=4,
  norm_trace=False, rank='full', reg='shrunk',
  transform_into='average_power')

In [2]:
import matplotlib.pyplot as plt
from mne.viz.topomap import _prepare_topo_plot
from mpl_toolkits.axes_grid1 import make_axes_locatable
_,epos,_,_,_ = _prepare_topo_plot(MI_epochs,'eeg',None)
# epos is the position of electrode
# plot first pattern
pattern = csp.patterns_[0,:]
pattern -= pattern.mean()
ix = np.argmax(abs(pattern))
# the parttern is sign invariant.
# invert it for display purpose
if pattern[ix]>0:
    sign = 1.0
else:
    sign = -1.0

fig, ax_topo = plt.subplots(1, 1, figsize=(12, 4))
title = 'Spatial Pattern'
fig.suptitle(title, fontsize=14)
img, _ = mne.viz.plot_topomap(sign*pattern,epos,axes=ax_topo,show=False)
divider = make_axes_locatable(ax_topo)
# add axes for colorbar
ax_colorbar = divider.append_axes('right', size='5%', pad=0.05)
plt.colorbar(img, cax=ax_colorbar)
# try spetral pattern
from scipy.signal import welch
po = []
for x in X:
    f,p = welch(np.dot(csp.filters_[0,:].T,x), 500, nperseg=512)
    po.append(p)
po = np.array(po)
fix = (f>0.5) & (f<59)
ax_spectrum = divider.append_axes('right', size='300%', pad=1.2)
ax_spectrum.plot(f[fix],np.log(po[y==1][:,fix].mean(axis=0).T),'-r',lw=2)
ax_spectrum.plot(f[fix],np.log(po[y==0][:,fix].mean(axis=0).T),'-b',lw=2)
ax_spectrum.set_xlabel('Frequency (Hz)')
ax_spectrum.set_ylabel('Power (dB)')
plt.grid()
plt.legend(['left','right'])
plt.title('ZZL')
plt.show()
plt.savefig('spatial_pattern_subject_zzl.png'  ,bbox_inches='tight')

<Figure size 1200x400 with 3 Axes>

In [30]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
clf = make_pipeline(CSP(n_components=5,reg='shrunk',cov_est='concat',rank ='full'),LogisticRegression())
auc=[]
auc.append(cross_val_score(clf,X,y,scoring='roc_auc').mean())
#auc of Logis is about 0.5

F:\software\2PYTHON\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Computing data rank from raw with rank='full'
    MAG: rank 32 from info
Reducing data rank from 32 -> 32
Estimating covariance using SHRUNK


F:\software\2PYTHON\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Done.
Computing data rank from raw with rank='full'
    MAG: rank 32 from info
Reducing data rank from 32 -> 32
Estimating covariance using SHRUNK


F:\software\2PYTHON\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


...

F:\software\2PYTHON\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Done.
Computing data rank from raw with rank='full'
    MAG: rank 32 from info
Reducing data rank from 32 -> 32
Estimating covariance using SHRUNK


F:\software\2PYTHON\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Done.


F:\software\2PYTHON\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [34]:
#Accuracy of training process
from  sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.3, random_state=42)

In [46]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.pipeline import Pipeline
lda = LinearDiscriminantAnalysis()
clf = Pipeline([('CSP', csp), ('LDA', lda)])
cv = ShuffleSplit(10, test_size=0.2, random_state=42)
scores = cross_val_score(clf, X_train, y_train, cv=cv, n_jobs=1)

Computing data rank from raw with rank='full'
    MAG: rank 32 from info
Reducing data rank from 32 -> 32
Estimating covariance using SHRUNK


F:\software\2PYTHON\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Done.
Computing data rank from raw with rank='full'
    MAG: rank 32 from info
Reducing data rank from 32 -> 32
Estimating covariance using SHRUNK


F:\software\2PYTHON\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Done.
Computing data rank from raw with rank='full'
    MAG: rank 32 from info
Reducing data rank from 32 -> 32
Estimating covariance using SHRUNK


...

Done.
Computing data rank from raw with rank='full'
    MAG: rank 32 from info
Reducing data rank from 32 -> 32
Estimating covariance using SHRUNK


F:\software\2PYTHON\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Done.
Computing data rank from raw with rank='full'
    MAG: rank 32 from info
Reducing data rank from 32 -> 32
Estimating covariance using SHRUNK


F:\software\2PYTHON\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Done.


In [3]:
from  sklearn.model_selection import train_test_split
csp = CSP(n_components=4,reg='shrunk',cov_est='concat',rank='full')
X_tr=csp.fit_transform(X,y)
X_train, X_test, y_train, y_test =train_test_split(X_tr, y, test_size=0.3, random_state=42)
lda = LinearDiscriminantAnalysis()
scores = cross_val_score(lda, X_tr,y, cv=5)
# lda.fit(X_train,y_train)
# lda.score(X_test,y_test)

Computing data rank from raw with rank='full'
    MAG: rank 32 from info
Reducing data rank from 32 -> 32
Estimating covariance using SHRUNK


F:\software\2PYTHON\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Done.
Computing data rank from raw with rank='full'
    MAG: rank 32 from info
Reducing data rank from 32 -> 32
Estimating covariance using SHRUNK


F:\software\2PYTHON\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Done.


NameError: name 'LinearDiscriminantAnalysis' is not defined

In [5]:
# Training accuracy
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score
lda = LinearDiscriminantAnalysis()
scores = cross_val_score(lda, X_tr,y, cv=5)
scores.mean()

0.7497994987468672

In [6]:
import mne
from mne.decoding import CSP
import pandas
import numpy as np
#Using locs to read montage
montage_NE = mne.channels.read_montage(kind='Enobio32',path=r'F:\software\1NE_NIC2\1channelL')
epochs_zzl2Session2 = mne.io.read_epochs_eeglab(input_fname=r'F:\software\1NE_NIC2\raw\MI10Oct_and8_oneCmc\zzl2Session2.set',
                             montage=montage_NE)
# %matplotlib qt
# epochs_zzl2Session1.plot()
MI_epochs = epochs_zzl2Session1.copy()
MI_epochs.crop(tmin = 3, tmax = 7)
# MI_epochs.plot()

y_tr = pandas.read_csv('F:/software/1NE_NIC2/raw/MI10Oct_and8_oneCmc/100times_test_section2.csv')
labels=y.iloc[1:101, 2]
y=np.array([])
for label in labels:
    if label == 'left':
        y=np.append(y,1)
    else:
        y=np.append(y,0)

X=MI_epochs.get_data()
csp = CSP(n_components=4,reg='shrunk',cov_est='concat',rank ='full')

X_trS2=csp.fit_transform(X,y)

Extracting parameters from F:\software\1NE_NIC2\raw\MI10Oct_and8_oneCmc\zzl2Session2.set...
100 matching events found
No baseline correction applied
Not setting metadata


<ipython-input-6-3aacc9f69f6b>:8: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  montage=montage_NE)


0 projection items activated
Ready.
Computing data rank from raw with rank='full'
    MAG: rank 32 from info
Reducing data rank from 32 -> 32
Estimating covariance using SHRUNK


F:\software\2PYTHON\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Done.
Computing data rank from raw with rank='full'
    MAG: rank 32 from info
Reducing data rank from 32 -> 32
Estimating covariance using SHRUNK


F:\software\2PYTHON\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Done.


In [19]:
# Used first exp as training data, second as test data
y = pandas.read_csv('F:/software/1NE_NIC2/raw/MI10Oct_and8_oneCmc/100times_test_section1.csv')
labels=y.iloc[1:101, 2]
y=np.array([])
for label in labels:
    if label == 'left':
        y=np.append(y,1)
    else:
        y=np.append(y,0)

        
y2 = pandas.read_csv('F:/software/1NE_NIC2/raw/MI10Oct_and8_oneCmc/100times_test_section2.csv')
labels=y2.iloc[1:101, 2]
y2=np.array([])
for label in labels:
    if label == 'left':
        y2=np.append(y2,1)
    else:
        y2=np.append(y2,0)
        
lda.fit(X_tr,y)
lda.score(X_trS2,y2)

0.55

In [35]:
# Mixing data set
X_total = np.vstack((X_tr,X_trS2))
y_total=np.append(y,y2)
X_train, X_test, y_train, y_test =train_test_split(X_total, y_total, test_size=0.3, random_state=42)
lda.fit(X_train,y_train)
lda.score(X_test,y_test)

0.6166666666666667